In [17]:
import pandas as pd
import numpy as np
import random

In [69]:
path = '../data/tyc/'
data = pd.read_csv(path + 'comp_touzi_comp.csv')

In [5]:
df = data.set_index(data['date'])
df = df.sort_index()

df

,src_cid,src_cname,src_ind,dst_cid,dst_cname,dst_ind,date
date,,,,,,,
1985-08-20,company/24212775,天津津融投资服务集团有限公司,28647,company/264022019,天津国际联合轮胎橡胶股份有限公司,31806,1985-08-20
1989-06-01,company/146170414,中冶建工集团有限公司,5168,company/7507939,中冶宝钢技术服务有限公司,60359,1989-06-01
1990-03-19,company/367314478,东证锦信投资管理有限公司,52725,company/6941042,虎彩印艺股份有限公司,59394,1990-03-19
1991-03-11,company/353779474,广东省广晟资产经营有限公司,52385,company/816472108,深圳市长城惠华集团有限公司,61898,1991-03-11
1991-05-01,company/25731673,江阴海澜创业投资有限公司,31192,company/3208046385,华泰证券股份有限公司,45762,1991-05-01
...,...,...,...,...,...,...,...
2020-11-30,company/3152144800,新余昆诺投资管理有限公司,43390,company/3224097797,北京十荟科技有限公司,46531,2020-11-30
2020-11-30,company/13027951,和才（天津）股权投资基金管理有限公司,3520,company/227664287,天津达因建材有限公司,9858,2020-11-30
2020-11-30,company/113671400,北京文资数码投资管理有限公司,1507,company/28702414,北京伟德杰生物科技有限公司,33360,2020-11-30


In [7]:
print(f'user.max: {data.src_ind.max()}, item.max: {data.dst_ind.max()}')

src_set, dst_set = set(data.src_ind), set(data.dst_ind)
print(f'#user: {len(src_set)}, #item: {len(dst_set)}, user & item: {len(src_set&dst_set)}, user | item: {len(src_set|dst_set)}')

user.max: 64422, item.max: 64425
#user: 4674, #item: 42068, user & item: 315, user | item: 46427


In [8]:
# print #records after date
def print_stat(dt):
    comps = set(df[dt:]['src_ind'])
    print(f'{dt}~: {len(comps)} comps, {len(df[dt:])} records')

In [12]:
print_stat('2020-01-01')
print_stat('2019-11-30')
print_stat('2019-06-30')
print_stat('2019-01-01')
print_stat('2018-01-01')

2020-01-01~: 1462 comps, 4570 records
2019-11-30~: 1614 comps, 5247 records
2019-06-30~: 2135 comps, 8022 records
2019-01-01~: 2731 comps, 12345 records
2018-01-01~: 3640 comps, 24176 records


In [48]:
# remove names
df_ = df[['src_ind', 'dst_ind', 'date']]
df_.insert(3, 'label', 1)

train_df = df_[:'2018-12-31']
valid_df = df_['2019-01-01':'2019-12-31']
test_df = df_['2020-01-01':]

train_df

,src_ind,dst_ind,date,label
date,,,,
1985-08-20,28647,31806,1985-08-20,1
1989-06-01,5168,60359,1989-06-01,1
1990-03-19,52725,59394,1990-03-19,1
1991-03-11,52385,61898,1991-03-11,1
1991-05-01,31192,45762,1991-05-01,1
...,...,...,...,...
2018-12-31,20602,54478,2018-12-31,1
2018-12-31,28716,6830,2018-12-31,1
2018-12-31,22944,32714,2018-12-31,1


In [50]:
def negative_sampling(df, dst_set, k=99):
    neg_src = np.repeat(df['src_ind'], k)
    dst = list(dst_set)
    neg_dst = [random.choice(dst) for _ in range(len(neg_src))]
    test_neg_df = pd.DataFrame({'src_ind': neg_src, 'dst_ind': neg_dst, 'label': 0})
    return test_neg_df

In [51]:
def negative_sampling_by_user(df, dst_set, k=99):
    neg_src = np.repeat(list(set(df['src_ind'])), k)
    dst = list(dst_set)
    neg_dst = [random.choice(dst) for _ in range(len(neg_src))]
    test_neg_df = pd.DataFrame({'src_ind': neg_src, 'dst_ind': neg_dst, 'label': 0})
    return test_neg_df

In [66]:
random.seed(523)
test_neg_df = negative_sampling(test_df, set(data.dst_ind))
test_neg_user_df = negative_sampling_by_user(test_df, set(data.dst_ind))

len(test_df), len(test_neg_df), len(test_neg_user_df)

(4570, 452430, 144738)

In [61]:
def intersect(df1: pd.DataFrame, df2):
    df1 = df1.set_index(['src_ind', 'dst_ind'])
    df2 = df2.set_index(['src_ind', 'dst_ind'])
    return df1.join(df2, how='inner', lsuffix='_pos', rsuffix='_neg')

In [67]:
intersect(test_df, test_neg_df)

date  label_pos  label_neg
src_ind dst_ind                                  
546     10034    2020-11-16          1          0
1255    53913    2020-08-25          1          0
1888    49509    2020-06-11          1          0
1907    57142    2020-08-05          1          0
3360    13725    2020-10-30          1          0
...                     ...        ...        ...
59124   46670    2020-10-16          1          0
60942   44034    2020-07-31          1          0
        49930    2020-01-01          1          0
61647   12075    2020-10-26          1          0
        26305    2020-08-18          1          0

[105 rows x 3 columns]

In [68]:
intersect(test_df, test_neg_user_df)

,,date,label_pos,label_neg
src_ind,dst_ind,,,
7418,20749,2020-04-30,1,0
10103,44430,2020-04-15,1,0
14594,57247,2020-01-06,1,0
14805,53566,2020-09-29,1,0
20189,13077,2020-01-03,1,0
20857,4213,2020-07-31,1,0
22914,51370,2020-08-25,1,0
25549,51919,2020-04-17,1,0
28913,48017,2020-09-04,1,0


In [70]:
train_df.to_csv(path + 'train.csv', index=False)
valid_df.to_csv(path + 'valid.csv', index=False)
test_df.to_csv(path + 'test.csv', index=False)
test_neg_df.to_csv(path + 'test_neg.csv', index=False)
test_neg_user_df.to_csv(path + 'test_neg_user.csv', index=False)